In [ ]:
import pandas as pd
import pandera as pa

In [ ]:
valores_ausentes = ['###!','****','***','####','*****','NULL'] #limpeza de dados no carregmento (extracao)

df = pd.read_csv("ocorrencia.csv",nrows=2000, sep=";", parse_dates=['ocorrencia_dia'], dayfirst=True, na_values= valores_ausentes)

df.head(10)

In [ ]:
schema = pa.DataFrameSchema(
columns={
    "codigo": pa.Column(pa.Int, required=False),
    "codigo_ocorrencia": pa.Column(pa.Int),
    "codigo_ocorrencia2":pa.Column(pa.Int),
    "ocorrencia_classificacao": pa.Column(pa.String),
    "ocorrencia_cidade":pa.Column(pa.String),
    "ocorrencia_cidade":pa.Column(pa.String),
    "ocorrencia_uf":pa.Column(pa.String, pa.Check.str_length(2,2), nullable=True),
    "ocorrencia_dia":pa.Column(pa.DateTime),
    "ocorrencia_hora":pa.Column(pa.String, pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5][0-9])(:[0-5][0-9])?$'),nullable=True),
    "total_recomendacoes":pa.Column(pa.Int)
    }
)

In [ ]:
schema.validate(df)

In [ ]:
df.dtypes

# FILTRANDO DADOS

In [ ]:
df.loc[1]

In [ ]:
df.iloc[1]#iloc busca pelo indice ou loc pelo label

In [ ]:
df.iloc[-1]

In [ ]:
df.tail()

In [ ]:
df.iloc[10:15]

In [ ]:
df.loc[:,'ocorrencia_uf']

In [ ]:
df['ocorrencia_uf']

In [ ]:
df.isna().sum() #quais colunas possuem valores nao informados ou isnull

In [ ]:
filtro =df.ocorrencia_aerodromo.isnull()

df.loc[filtro]

In [ ]:
df.count() #contagem 

In [ ]:
# filtrar numeros ocorrencias com mais de 10 recomendacoes
filtro = df.total_recomendacoes > 10
df.loc[filtro]

In [ ]:
filtro = df.total_recomendacoes > 10
df.loc[filtro, ['ocorrencia_cidade', 'total_recomendacoes']]

In [ ]:
#ocorrencias com classificao incidente grave
filtro = df.ocorrencia_classificacao == 'INCIDENTE GRAVE'
df.loc[filtro, ['ocorrencia_cidade', 'total_recomendacoes','ocorrencia_classificacao']]

In [ ]:
#combinar dois filtros
#ocorrencias com classificao incidente grave e estado é SP se quiser ou é (|)

filtro1 = df.ocorrencia_classificacao == 'INCIDENTE GRAVE'
filtro2 = df.ocorrencia_uf == 'SP'
df.loc[filtro1 & filtro2,['ocorrencia_uf', 'ocorrencia_classificacao']]

In [ ]:
#metodo isin(['INCIDENTE GRAVE','INCIDENTE']) = Verifica se esta dentro das possibilidader melhor que usar operar OU |
filtro1 = df.ocorrencia_classificacao.isin(['INCIDENTE GRAVE','INCIDENTE'])
filtro2 = df.ocorrencia_uf == 'SP'
df.loc[filtro1 & filtro2]

In [ ]:
# procurar todas as cidades começãm com a letra C
filtro = df.ocorrencia_cidade.str[0] == 'C'
df.loc[filtro]

In [ ]:
# procurar todas as cidades terminam com a letra A
filtro = df.ocorrencia_cidade.str[0] == 'A'
df.loc[filtro]

In [ ]:
# procurar todas as cidades terminam com as letras MA
filtro = df.ocorrencia_cidade.str[-2:] == 'MA'
df.loc[filtro]

In [ ]:
#  filtrar strings procurar todas as cidades contem em qualquer parte com as letras MA pode usar tambem or |
filtro = df.ocorrencia_cidade.str.contains('MA|AL')
df.loc[filtro]

In [ ]:
#filtrar datas ocorrencias do ano de 2011
filtro = df.ocorrencia_dia.dt.year == 2011
df.loc[filtro]

In [ ]:
#filtrar datas ocorrencias do ano de 2011 mes 12 para o dia só usar .day
#(filtro = df.ocorrencia_dia.dt.year == 2011) & (filtro2 =df.ocorrencia_dia.dt.month == 12)
filtro1 = df.ocorrencia_dia.dt.year == 2011
filtro2 = df.ocorrencia_dia.dt.month == 12
df.loc[filtro1 & filtro2]

In [ ]:
#f iltrar do dia 3 ao dia 8 só colocar dia (.day>=3) & (df.ocorrencia_dia.dt.day <=8)

In [ ]:
#transformar a coluna data e hora em uma só ( str + date) tem que tranformar dia em str e depois pd.to_datetime para tranformar em data
df['ocorrencia_dia_hora'] = pd.to_datetime(df.ocorrencia_dia.astype(str) + ' ' + df.ocorrencia_hora)

In [ ]:
df.head()

In [ ]:
filtro1 = df.ocorrencia_dia_hora >= '2011-12-03 11:00:00'
filtro2 = df.ocorrencia_dia_hora <= '2011-12-08 13:00:00'
df[filtro1 & filtro2]

# AGRUPANDO DADOS

In [ ]:
#filtrar datas ocorrencias do ano de 2011 E MÊS 3
filtro1 = df.ocorrencia_dia.dt.year == 2011
filtro2 = df.ocorrencia_dia.dt.month == 2
df201102 = df.loc[filtro1 & filtro2]

In [ ]:
df201102

In [ ]:
df201102.count()

In [ ]:
#contando pelo codigo de ocorrencia sempre verficar qual a melhor coluna para se contar se usa geralemtne a coluna que possui todos os dados
df201102.groupby(['ocorrencia_classificacao']).codigo_ocorrencia.count()

In [ ]:
df201102.groupby(['ocorrencia_classificacao']).ocorrencia_aerodromo.count()

In [ ]:
#contar por size = contar o numero de linhas que foram agrupadas
df201102.groupby(['ocorrencia_classificacao']).size()

In [ ]:
#ordenar
df201102.groupby(['ocorrencia_classificacao']).size().sort_values()

In [ ]:
df201102.groupby(['ocorrencia_classificacao']).size().sort_values(ascending=False)

In [ ]:
#agrupar ocorrencias no ano de 2011 na regiao sudeste 
filtro1 = df.ocorrencia_dia.dt.year == 2011
filtro2 = df.ocorrencia_uf.isin(['SP','MG','ES','RJ'])
dfsudeste2011 = df.loc[filtro1 & filtro2]
dfsudeste2011

In [ ]:
dfsudeste2011.groupby(['ocorrencia_classificacao']).size()

In [ ]:
dfsudeste2011.groupby(['ocorrencia_classificacao','ocorrencia_uf']).size()

In [ ]:
dfsudeste2011.groupby(['ocorrencia_uf','ocorrencia_classificacao']).size()

In [ ]:
dfsudeste2011.groupby(['ocorrencia_cidade']).size().sort_values(ascending=False)

In [ ]:
filtro = dfsudeste2011.ocorrencia_cidade == 'RIO DE JANEIRO'
dfsudeste2011[filtro]

In [ ]:
#total de recomendacoes que o RJ teve
dfsudeste2011[filtro].total_recomendacoes.sum()

In [ ]:
dfsudeste2011.groupby(['ocorrencia_cidade']).total_recomendacoes.sum()

In [ ]:
filtro1 = dfsudeste2011.ocorrencia_cidade == 'RIO DE JANEIRO'
filtro2 = dfsudeste2011.total_recomendacoes > 0
dfsudeste2011.loc[filtro1 & filtro2]

In [ ]:
dfsudeste2011.groupby(['ocorrencia_aerodromo']).total_recomendacoes.sum()

In [ ]:
#dropna traz algo que nao foi informado nas colunas, ele traz para o agrupamento
dfsudeste2011.groupby(['ocorrencia_aerodromo'], dropna=False).total_recomendacoes.sum()

In [ ]:
filtro = dfsudeste2011.total_recomendacoes > 0
dfsudeste2011.loc[filtro].groupby(['ocorrencia_cidade']).total_recomendacoes.sum().sort_values(ascending=False)

In [ ]:

dfsudeste2011.loc[filtro].groupby(['ocorrencia_cidade', dfsudeste2011.ocorrencia_dia.dt.month]).total_recomendacoes.sum().sort_values(ascending=False)

In [ ]:
filtro1 = dfsudeste2011.total_recomendacoes > 0
filtro2 = dfsudeste2011.ocorrencia_cidade == 'SÃO PAULO'
dfsudeste2011.loc[filtro1 & filtro2]